In [1]:
import trimesh as tm
import numpy as np
import pyvista as pv
import os
import pickle as pk
import topogenesis as tg
from datetime import date
from ladybug.sunpath import Sunpath
from ladybug.epw import EPW
import math

In [2]:
context_path = os.path.relpath('../../data/movedcontext.obj')
context_mesh = tm.load(context_path)

env_lat_path = os.path.relpath('../../data/macrovoxels.csv')
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [3]:
# daylighting requirements - for now: put at march 21 - sept 21, 2 hrs of sunlight (?)

# location:
Longtitude = 4.3571
Lattitude = 52.0116

start_month = 3
start_day = 21

end_month = 9
end_day = 21
# minimum sunlight hours per day for this period
min_hours = 2

start_doy = date(2014, start_month, start_day).timetuple().tm_yday
end_doy = date(2014, end_month, end_day).timetuple().tm_yday

################### OLD ###########################

# def hoystart(doy):
#     hoycurr = (doy-1) * 24
#     return hoycurr

# def hoyend(doy):
#     hoycurr = doy * 24
#     return hoycurr

# def calc_hoys(start, end, step):
#     first_hoy = hoystart(start)
#     last_hoy = hoyend(end)
#     if first_hoy < last_hoy:
#         return list(range(first_hoy, last_hoy, step)) 
#     elif first_hoy > last_hoy:
#         a = list(range(0, last_hoy, step))
#         b = list(range(first_hoy, 365*24, step))
#         return a + b
#     else:
#         return None

#################### END OF OLD ###################

In [4]:
# assign illuminance and irradiance values to the vectors
# path1 = os.path.abspath("C:\Users\maxke\Documents\GitHub\thesis_Max_Ketelaar\data\NLD_Amsterdam.062400_IWEC.epw")
path = os.path.relpath("../../input/NLD_Amsterdam.062400_IWEC.epw")
epw = EPW(path)

dnr = epw.direct_normal_radiation
dni = epw.direct_normal_illuminance
ghi = epw.global_horizontal_illuminance

# include sun hoy azimuth, PV tilt, 

In [5]:
# sp = Sunpath(longtitude=Longtitude, lattitude=Lattitude)
sp = Sunpath(longitude=4.3571, latitude=52.0116)
# seems to only work like this

sunvectors = []
hoys = []
dnrval = []
dnival = []
ghival = []

# hours = calc_hoys(start_doy, end_doy, 1)
# for hoy in hours:
#     sun = sp.calculate_sun_from_hoy(hoy)
#     sun_vec = sun.sun_vector.to_array()
#      # remove vectors under horizon
#     if sun_vec[2] < 0.0:
#         hoys.append(hoy)
#         sunvectors.append(sun_vec)
#     # this calculates EVERY hoy for the given period

for i in range(0, 365, 7): # gives weekly moments
    for j in range(24): # can also give a range of f.e. 12:00-14:00 hrs if needed
        k = i*24 + j
        sun = sp.calculate_sun_from_hoy(k)
        sun_vec = sun.sun_vector.to_array()
        if sun_vec[2] < 0.0:
            hoys.append(j)
            sunvectors.append(sun_vec)
            dnrval.append(dnr[k])
            dnival.append(dni[k])
            ghival.append(ghi[k])
            # include dni and dnr and ghi and azimuth
            # azimuth = sun.altitude # see documentation from ladybug
            # azi = sun.azimuth
            
# convert to numpy array
sunvectors = np.array(sunvectors)
hoys = np.array(hoys)
dnrval = np.array(dnrval)
dnival = np.array(dnival)
ghival = np.array(ghival)

# rotate vectors to correct orientation for site
# TODO: check correct rotation for site --> 116 or 36???

rotation = 36.324
Rz = tm.transformations.rotation_matrix(np.radians(rotation), [0,0,1])
sunvectors = tm.transform_points(sunvectors, Rz)

# print(sunvectors.shape)
# sunvectors[1]
# x,y,z = sunvectors[1][0], sunvectors[1][1], sunvectors[1][2]

# magnitude = math.sqrt(x**2 + y**2 + z**2)
# unitvector = (x/magnitude, y/magnitude, z/magnitude)
# unitvector, (x,y,z)

In [38]:
# save global horizontal illuminance to pk
pk.dump(ghival, open("../../data/ghival.pk", "wb"))

In [39]:
# save hoys  to pk
pk.dump(hoys, open("../../data/hoys.pk", "wb"))

In [40]:
# save sun vectors to pk
pk.dump(sunvectors, open("../../data/sunvectors.pk", "wb"))

In [41]:
# save direct normal radiation to pk
pk.dump(dnrval, open("../../data/dnrval.pk", "wb"))

In [42]:
# save direct normal illuminance to pk
pk.dump(dnival, open("../../data/dnival.pk", "wb"))

In [6]:
a = - sunvectors * 300
b = np.array((30,-30,0))
vec = b - a

In [7]:
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

# add the sky vectors
# p.add_points(- sunvectors * 300 , color='#0013ff')
p.add_arrows(- sunvectors * 300, vec, mag=0.1, show_scalar_bar=False)

# add context
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, color='lightgrey')

# plotting
p.show(use_ipyvtk=True, screenshot='sunpath.png')

# TODO: check the sunpath points/locations as well as environment: might be something off about z-values!

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(981.9440024651442, 977.4066642083083, 1112.6340474642668),
 (-0.1175537109375, -4.6548919677734375, 130.57249128818512),
 (0.0, 0.0, 1.0)]

In [45]:
# calculate yield from surface normal vector (by defining rotation and tilt of panels) and Solar radiation perpendicular to sun and the solar vectors from GH
# Smodule = SincidentS.N where Sincident is solar rad perp to sun, S is unit vector to sun, N is unit vector normal to surface
# from: https://www.pveducation.org/pvcdrom/properties-of-sunlight/arbitrary-orientation-and-tilt 

azimuth_angle = 10 # degrees (counterclockwise) the panels are rotated from north
azimuth = math.radians(azimuth_angle + 180) # northern hemisphere

pv_angle = 70 # degrees of tilt for the pv panels on the roof (0 = horizontal, 90 = vertical)
pv_tilt = math.radians(pv_angle)

# calculate xyz components of surface (plane)
Vx, Vy, Vz = math.sin(pv_tilt) * math.sin(azimuth), -math.sin(pv_tilt) * math.cos(azimuth), math.cos(pv_tilt)

# check if it is a unit vector
# magnitude  = math.sqrt( Vx**2 + Vy**2 + Vz**2)
# unit_vector  = ( Vx / magnitude ,  Vy / magnitude,  Vz / magnitude )
# unit_vector, (Vx, Vy, Vz)

In [46]:
# calculating actual annual usage
# test for a single hoy, on the previous panel, using a random ray:
# using: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 
# E = Energy (kWh)
# A = Total solar panel Area (m2)
# r = solar panel yield or efficiency(%) - 15% or something
# H = Annual average solar radiation on tilted panels (shadings not included) = Smodule over the entire year
# PR = Performance ratio, coefficient for losses (range between 0.5 and 0.9, default value = 0.75)

# currently gives unexpected and sometimes negative values, possibly angle and/or azimuth are formulated wrong. EPW file also seems to contain unexpected data

hoy = 62
testsun = sp.calculate_sun_from_hoy(hoy)
testsun_vec = testsun.sun_vector.to_array()

Sincident = dnr[hoy]
N = Vx, Vy, Vz
S = testsun_vec

Smodule = Sincident * np.dot(S, N)
Smodule, (N), Sincident, S

(194.4476027561066,
 (-0.16317591116653493, 0.9254165783983233, 0.3420201433256688),
 290,
 (0.444711558077882, 0.8745056110557492, -0.19357573800441288))